###Exploring the data 

In [1]:

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pandas as pd;
import numpy as np;
from fuzzywuzzy import fuzz;
from nltk.stem.porter import *;
from nltk.stem.snowball import *;

stemmer = PorterStemmer();

C:\Users\sotripathi\AppData\Local\Dato\Dato Launcher\lib\site-packages\fuzzywuzzy\fuzz.py:33: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [2]:
path = {
 'data' : 'C:/Users/sotripathi/Desktop/Pypro/data/',       
 'submission' : 'C:/Users/sotripathi/Desktop/Pypro/submission/', 
 'python' : 'C:/Users/sotripathi/Desktop/Pypro/python/' 
   }

In [3]:
train = pd.read_csv(path['data']+ 'train.csv',encoding="ISO-8859-1")
test = pd.read_csv(path['data']+ 'test.csv',encoding="ISO-8859-1")
submission = pd.read_csv(path['data']+ 'sample_submission.csv')
attributes = pd.read_csv(path['data']+ 'attributes.csv',encoding="ISO-8859-1")
product_descriptions = pd.read_csv(path['data']+ 'product_descriptions.csv',encoding="ISO-8859-1")

In [4]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


###Findings from the initial analysis & ideas
    * check for fuzzy match
    * from the first few examples it looks like, relevance as 1 if no words found in the data - even though it goes against  whats mentioned in the relevance instructions

In [5]:
#creating columns from fuzzy match
def fuz_columns(x,y):
    return fuzz.token_set_ratio(x,y)     #.apply(lambda x1,x2 : fuzzy )

#creating columns from fuzzy match
def fuz_columns_sort(x,y):
    
    return fuzz.token_sort_ratio(x,y)

#creating columns from fuzzy match
def fuz_columns_pr(x,y):
    return fuzz.partial_ratio(x,y)


In [6]:
train.head()

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67


Adding other variables from uploaded scripts

In [13]:
def str_stem(s): 
    if isinstance(s, unicode):
        s = s.lower()
        s = s.replace("'","in.") # character
        s = s.replace("inches","in.") # whole word
        s = s.replace("inch","in.") # whole word
        s = s.replace(" in ","in. ") # no period
        s = s.replace(" in.","in.") # prefix space

        s = s.replace("''","ft.") # character
        s = s.replace(" feet ","ft. ") # whole word
        s = s.replace("feet","ft.") # whole word
        s = s.replace("foot","ft.") # whole word
        s = s.replace(" ft ","ft. ") # no period
        s = s.replace(" ft.","ft.") # prefix space
    
        s = s.replace(" pounds ","lb. ") # character
        s = s.replace(" pound ","lb. ") # whole word
        s = s.replace("pound","lb.") # whole word
        s = s.replace(" lb ","lb. ") # no period
        s = s.replace(" lb.","lb.") 
        s = s.replace(" lbs ","lb. ") 
        s = s.replace("lbs.","lb.") 
    
        s = s.replace("*"," xby ")
        s = s.replace(" by"," xby")
        s = s.replace("x0"," xby 0")
        s = s.replace("x1"," xby 1")
        s = s.replace("x2"," xby 2")
        s = s.replace("x3"," xby 3")
        s = s.replace("x4"," xby 4")
        s = s.replace("x5"," xby 5")
        s = s.replace("x6"," xby 6")
        s = s.replace("x7"," xby 7")
        s = s.replace("x8"," xby 8")
        s = s.replace("x9"," xby 9")
    
        s = s.replace(" sq ft","sq.ft. ") 
        s = s.replace("sq ft","sq.ft. ")
        s = s.replace("sqft","sq.ft. ")
        s = s.replace(" sqft ","sq.ft. ") 
        s = s.replace("sq. ft","sq.ft. ") 
        s = s.replace("sq ft.","sq.ft. ") 
        s = s.replace("sq feet","sq.ft. ") 
        s = s.replace("square feet","sq.ft. ") 
    
        s = s.replace(" gallons ","gal. ") # character
        s = s.replace(" gallon ","gal. ") # whole word
        s = s.replace("gallons","gal.") # character
        s = s.replace("gallon","gal.") # whole word
        s = s.replace(" gal ","gal. ") # character
        s = s.replace(" gal","gal") # whole word

        s = s.replace("°","deg. ")
        s = s.replace("degrees","deg. ")
        s = s.replace("degree","deg. ")
        
        s = s.replace("volts","volt. ")
        s = s.replace("volt","volt. ")

        s = s.replace("watts","watt. ")
        s = s.replace("watt","watt. ")

        s = s.replace("ampere","amp. ")
        s = s.replace("amps","amp. ")
        s = s.replace(" amp ","amp. ")
        
        s = s.replace("whirpool","whirlpool")
        s = s.replace("whirlpoolga", "whirlpool")
        s = s.replace("whirlpoolstainless","whirlpool stainless")

        
        
        s = s.replace(" ounces","oz.")
        s = s.replace(" ounce","oz.")
        s = s.replace("ounce","oz.")
        s = s.replace(" oz ","oz. ")

        s = s.replace(" centimeters","cm.")    
        s = s.replace(" cm.","cm.")
        s = s.replace(" cm ","cm. ")
        
        s = s.replace(" milimeters","mm.")
        s = s.replace(" mm.","mm.")
        s = s.replace(" mm ","mm. ")
    
        s = (" ").join([stemmer.stem(z) for z in s.split(" ")])
        #volts, watts, amps
        return s.lower()
    else:
        return "null"
    
def str_common_word(str1, str2):
    words, cnt = str1.split(), 0
    for word in words:
        if str2.find(word)>=0:
            cnt+=1
    return cnt

def str_whole_word(str1, str2, i_):
    cnt = 0
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return cnt
        else:
            cnt += 1
            i_ += len(str1)
    return cnt

In [14]:
df_brand = attributes[attributes.name == "MFG Brand Name"][["product_uid", "value"]].rename(columns={"value": "brand"})
df_all = pd.concat((train, test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, product_descriptions, how='left', on='product_uid')
df_all = pd.merge(df_all, df_brand, how='left', on='product_uid')

In [41]:
product_descriptions[product_descriptions.product_uid==100001]

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."


In [21]:
isinstance((df_all['search_term'][1]),unicode)
#type(df_all['search_term'][1])
#unicodedata.normalize('NFKD', title).encode('ascii','ignore')
df_all

,id,product_title,product_uid,relevance,search_term,product_description,brand,len_of_query,len_of_title,len_of_description,len_of_brand
0,2,simpson strong-ti 12-gaug angl,100001,3.00,angl bracket,"not onli do angl make joint stronger, they als...",simpson strong-ti,2,4,124,2
1,3,simpson strong-ti 12-gaug angl,100001,2.50,l bracket,"not onli do angl make joint stronger, they als...",simpson strong-ti,2,4,124,2
2,9,behr premium textur deckov 1-gal. #sc-141 tugb...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,2,11,161,4
3,16,delta vero 1-handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero single...,delta,3,12,101,1
4,17,delta vero 1-handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero single...,delta,3,12,101,1
5,18,whirlpool 1.9 cu.ft. over the rang convect mic...,100006,3.00,convect otr,achiev delici result is almost effortless with...,whirlpool,2,13,488,1
6,20,whirlpool 1.9 cu.ft. over the rang convect mic...,100006,2.67,microwav over stove,achiev delici result is almost effortless with...,whirlpool,3,13,488,1
7,21,whirlpool 1.9 cu.ft. over the rang convect mic...,100006,3.00,microwav,achiev delici result is almost effortless with...,whirlpool,1,13,488,1
8,23,lithonia light quantum 2-light black led emerg...,100007,2.67,emerg light,the quantum adjust 2-light led black emerg lig...,lithonia light,2,9,120,2
9,27,hous of fara 3/4in. x 3in. x 8ft. mdf flute case,100009,3.00,mdf 3/4,get the hous of fara 3/4in. x 3in. x 8ft. mdf ...,hous of fara,2,11,78,3


In [15]:
#attributes 
#product_descriptions 

df_all['search_term'] = df_all['search_term'].map(lambda x:str_stem(x))

df_all['product_title'] = df_all['product_title'].map(lambda x:str_stem(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stem(x))

In [16]:
df_all['brand'] = df_all['brand'].map(lambda x:str_stem(x))
df_all['len_of_query'] = df_all['search_term'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_title'] = df_all['product_title'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_description'] = df_all['product_description'].map(lambda x:len(x.split())).astype(np.int64)
df_all['len_of_brand'] = df_all['brand'].map(lambda x:len(x.split())).astype(np.int64)

In [52]:
#from nltk.stem.snowball import SnowballStemmer
#stemmer = SnowballStemmer("english")
#stemmer.stem("plot")

In [53]:
df_all['token_sr_matchh'] = np.vectorize(fuz_columns)(np.array(df_all['search_term']),np.array(df_all['product_title']))
df_all['token_sort_ratio'] = np.vectorize(fuz_columns_sort)(np.array(df_all['search_term']),np.array(df_all['product_title']))
df_all['partial_ratio'] = np.vectorize(fuz_columns_pr)(np.array(df_all['search_term']),np.array(df_all['product_title']))
df_all['pdesc_token_sr_matchh'] = np.vectorize(fuz_columns)(np.array(df_all['search_term']),
                                                            np.array(df_all['product_description']))
df_all['pdesc_token_sort_ratio'] = np.vectorize(fuz_columns_sort)(np.array(df_all['search_term']),
                                                                  np.array(df_all['product_description']))
df_all['pdesc_partial_ratio'] = np.vectorize(fuz_columns_pr)(np.array(df_all['search_term']),
                                                             np.array(df_all['product_description']))
df_all['br_token_sr_matchh'] = np.vectorize(fuz_columns)(np.array(df_all['search_term']),np.array(df_all['brand']))
df_all['br_token_sort_ratio'] = np.vectorize(fuz_columns_sort)(np.array(df_all['search_term']),np.array(df_all['brand']))
df_all['br_partial_ratio'] = np.vectorize(fuz_columns_pr)(np.array(df_all['search_term']),np.array(df_all['brand']))

In [56]:
df_all['product_info'] = df_all['search_term']+'\t'+df_all['product_title'] +'\t'+df_all['product_description']
df_all['query_in_title'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[1],0))

In [74]:
df_all.head()
num_train = train.shape[0]

In [58]:

df_all['query_in_description'] = df_all['product_info'].map(lambda x:str_whole_word(x.split('\t')[0],x.split('\t')[2],0))
df_all['word_in_title'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['word_in_description'] = df_all['product_info'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[2]))
df_all['ratio_title'] = df_all['word_in_title']/df_all['len_of_query']
df_all['ratio_description'] = df_all['word_in_description']/df_all['len_of_query']
df_all['attr'] = df_all['search_term']+"\t"+df_all['brand']
df_all['word_in_brand'] = df_all['attr'].map(lambda x:str_common_word(x.split('\t')[0],x.split('\t')[1]))
df_all['ratio_brand'] = df_all['word_in_brand']/df_all['len_of_brand']
df_brand = pd.unique(df_all.brand.ravel())

In [59]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description,brand,len_of_query,len_of_title,len_of_description,...,br_partial_ratio,product_info,query_in_description,word_in_title,word_in_description,ratio_title,ratio_description,attr,word_in_brand,ratio_brand
0,2,simpson strong-ti 12-gaug angl,100001,3.00,angl bracket,"not onli do angl make joint stronger, they als...",simpson strong-ti,2,4,124,...,33,angl bracket\tsimpson strong-ti 12-gaug angl\t...,0,1,1,0.500000,0.500000,angl bracket\tsimpson strong-ti,0,0.00
1,3,simpson strong-ti 12-gaug angl,100001,2.50,l bracket,"not onli do angl make joint stronger, they als...",simpson strong-ti,2,4,124,...,33,l bracket\tsimpson strong-ti 12-gaug angl\tnot...,0,1,1,0.500000,0.500000,l bracket\tsimpson strong-ti,0,0.00
2,9,behr premium textur deckov 1-gal. #sc-141 tugb...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,2,11,161,...,80,deck over\tbehr premium textur deckov 1-gal. #...,0,1,1,0.500000,0.500000,deck over\tbehr premium textur deckov,1,0.25
3,16,delta vero 1-handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero single...,delta,3,12,101,...,33,rain shower head\tdelta vero 1-handl shower on...,0,1,1,0.333333,0.333333,rain shower head\tdelta,0,0.00
4,17,delta vero 1-handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero single...,delta,3,12,101,...,44,shower onli faucet\tdelta vero 1-handl shower ...,0,3,2,1.000000,0.666667,shower onli faucet\tdelta,0,0.00


In [75]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]

In [83]:
d={}
i = 1
for s in df_brand:
    d[s]=i
    i+=1

df_all['brand_feature'] = df_all['brand'].map(lambda x:d[x])
df_all['search_term_feature'] = df_all['search_term'].map(lambda x:len(x))
#df_all.to_csv('df_all.csv')

In [104]:
df_all.to_csv(path['data']+'df_all.csv',encoding="ISO-8859-1")
#path['data']+'df_all.csv'

In [119]:
df_all = df_all.drop(['search_term','product_title','product_description','product_info','attr','brand'],axis=1)
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
ytrain = df_train['relevance'].values
Xtrain = df_train.drop(['id','relevance'],axis=1).values
Xtest = df_test.drop(['id','relevance'],axis=1).values

**XGBOOST - as a regrssion problem**

In [120]:
ytrain

array([ 3.  ,  2.5 ,  3.  , ...,  2.33,  3.  ,  2.33])

In [127]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import xgboost as xgb

# -*- coding: utf-8 -*-
"""
Created on Sat Jan 30 16:57:10 2016

@author: sotripathi
https://github.com/hyperopt/hyperopt/wiki/FMin - Hyperopt documentation
"""
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss
from sklearn import preprocessing

import numpy as np
import pandas as pd

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

#import sys
# The path to XGBoost wrappers goes here
#sys.path.append('C:\\Users\\Amine\\Documents\\GitHub\\xgboost\\wrapper')
import xgboost as xgb

def score(params):
    print "Training with params : "
    print params
    num_round = int(params['n_estimators'])
    del params['n_estimators']
    dtrain = xgb.DMatrix(X_train, y_train)
    dvalid = xgb.DMatrix(X_test, y_test)
    # watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
    model = xgb.train(params, dtrain, num_round)
    predictions = model.predict(dvalid)#.reshape((X_test.shape[0], 3))
    score = mean_squared_error(y_test, predictions)**0.5
    print "\tScore {0}\n\n".format(score)
    return {'loss': score, 'status': STATUS_OK}


def optimize(trials):
    space = {
             'n_estimators' : hp.quniform('n_estimators', 100, 1000, 5),
             'eta' : hp.quniform('eta', 0.025, 0.5, 0.025),
             'max_depth' : hp.quniform('max_depth', 1, 13, 1),
             'min_child_weight' : hp.quniform('min_child_weight', 1, 6, 1),
             'subsample' : hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma' : hp.quniform('gamma', 0.5, 1, 0.05),
             'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
             'eval_metric': 'rmse',
             'objective': 'reg:linear',
             'nthread' : 5,
             'silent' : 1,
             'alpha':hp.quniform('alpha', 0, 1, 0.05),
             'lambda':hp.quniform('lambda',0,1,0.05)
             }

    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=250)

    print best


X, y = Xtrain,ytrain
print "Splitting data into train and valid ...\n\n"
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=1234)

#Trials object where the history of search will be stored
trials = Trials()

optimize(trials)

'''
{'colsample_bytree': 0.65, 'min_child_weight': 1.0, 'n_estimators': 204.0,
 'subsample': 0.9500000000000001, 'eta': 0.17500000000000002, 'max_depth': 5.0, 'gamma': 0.75}
 '''


Splitting data into train and valid ...


Training with params : 
{'colsample_bytree': 0.8500000000000001, 'silent': 1, 'eval_metric': 'rmse', 'nthread': 5, 'min_child_weight': 2.0, 'n_estimators': 360.0, 'subsample': 0.8500000000000001, 'eta': 0.5, 'objective': 'reg:linear', 'alpha': 0.5, 'max_depth': 8.0, 'gamma': 0.7000000000000001, 'lambda': 0.7000000000000001}
	Score 0.248770489168


Training with params : 
{'colsample_bytree': 0.7000000000000001, 'silent': 1, 'eval_metric': 'rmse', 'nthread': 5, 'min_child_weight': 4.0, 'n_estimators': 770.0, 'subsample': 0.55, 'eta': 0.35000000000000003, 'objective': 'reg:linear', 'alpha': 0.30000000000000004, 'max_depth': 6.0, 'gamma': 0.65, 'lambda': 0.4}
	Score 0.253780070737


Training with params : 
{'colsample_bytree': 0.7000000000000001, 'silent': 1, 'eval_metric': 'rmse', 'nthread': 5, 'min_child_weight': 4.0, 'n_estimators': 150.0, 'subsample': 0.75, 'eta': 0.275, 'objective': 'reg:linear', 'alpha': 0.6000000000000001, 'max_depth': 3.0,

"\n{'colsample_bytree': 0.65, 'min_child_weight': 1.0, 'n_estimators': 204.0,\n 'subsample': 0.9500000000000001, 'eta': 0.17500000000000002, 'max_depth': 5.0, 'gamma': 0.75}\n "

In [136]:
params = {}
tot_ce=[]

param ={
    'colsample_bytree': 0.5,
    'min_child_weight': 6.0, 
    'n_estimators': 630.0,
    'subsample': 0.9500000000000001,
    'eta': 0.025, 
    'max_depth': 13.0, 
    'gamma': 0.6000000000000001,
    'eval_metric': 'rmse',
    'objective': 'reg:linear',
    'nthread' : 5,
    'silent' : 1,
    'lambda': 0.8500000000000001,
    'alpha' : 0.75
}
   
dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_test, y_test)    

num_round = int(param['n_estimators'])
del param['n_estimators']
plst = list(param.items())

model = xgb.train(plst, dtrain, num_round)
predictions = model.predict(dvalid)#.reshape((X_test.shape[0], 3))
score = mean_squared_error(y_test, predictions)**0.5
    
    

In [157]:
score**0.5

0.45567846985932514

In [138]:
d_testing = xgb.DMatrix(Xtest)
prediction_output = model.predict(d_testing)

In [151]:
prediction_output[prediction_output>3]

array([], dtype=float32)

In [153]:
otp=pd.DataFrame(prediction_output)
otp.to_csv(path['data']+'output.csv')
#.to_csv(path[3+'output.csv')

In [131]:

###Fix this
def write_submission(preds, output):
    sample = pd.read_csv('C:\Users\sotripathi\Desktop\AV\\test_knn.csv')
    preds = pd.DataFrame(
        preds, index=sample.ID.values, columns=sample.columns[1:])
    preds.to_csv(output, index_label='ID')

{'book_time': None,
 'exp_key': None,
 'misc': {'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'idxs': {'alpha': [216],
   'colsample_bytree': [216],
   'eta': [216],
   'gamma': [216],
   'lambda': [216],
   'max_depth': [216],
   'min_child_weight': [216],
   'n_estimators': [216],
   'subsample': [216]},
  'tid': 216,
  'vals': {'alpha': [0.75],
   'colsample_bytree': [0.5],
   'eta': [0.025],
   'gamma': [0.6000000000000001],
   'lambda': [0.8500000000000001],
   'max_depth': [13.0],
   'min_child_weight': [6.0],
   'n_estimators': [630.0],
   'subsample': [0.9500000000000001]},
  'workdir': None},
 'owner': None,
 'refresh_time': None,
 'result': {'loss': 0.20764286789333591, 'status': 'ok'},
 'spec': None,
 'state': 2,
 'tid': 216,
 'version': 0}

In [158]:
xgb.plot_importance(model)

In [ ]:
prediction_output = model.predict(dtrain[])

In [178]:
d_test_rand = xgb.DMatrix(Xtrain[16:17])
prediction_output_rand = model.predict(d_test_rand)
prediction_output_rand

array([ 2.681566], dtype=float32)

In [180]:
#ytrain[16:17]
2 in [3]

False

In [188]:
df_train[df_train.relevance == [2.5]]

,id,product_uid,relevance,len_of_query,len_of_title,len_of_description,len_of_brand,token_sr_matchh,token_sort_ratio,partial_ratio,...,br_partial_ratio,query_in_description,word_in_title,word_in_description,ratio_title,ratio_description,word_in_brand,ratio_brand,brand_feature,search_term_feature
1,3,100001,2.5,2,4,124,2,10,10,20,...,33,0,1,1,0.500000,0.500000,0,0.000000,1,9
61,210,100037,2.5,3,4,64,1,53,48,50,...,100,0,1,2,0.333333,0.666667,1,1.000000,24,14
967,3039,100532,2.5,2,16,298,1,53,12,64,...,55,0,1,1,0.500000,0.500000,0,0.000000,14,11
1182,3690,100646,2.5,1,9,43,2,20,20,63,...,25,0,0,0,0.000000,0.000000,0,0.000000,29,8
1501,4606,100794,2.5,2,9,72,1,20,18,45,...,38,0,0,0,0.000000,0.000000,0,0.000000,87,11
1694,5182,100899,2.5,1,4,43,1,100,36,100,...,100,4,1,1,1.000000,1.000000,1,1.000000,177,4
13786,42619,110308,2.5,3,10,210,3,35,35,53,...,56,0,1,2,0.333333,0.666667,1,0.333333,1217,17
20043,61804,116699,2.5,3,6,76,2,80,45,58,...,57,0,2,2,0.666667,0.666667,1,0.500000,5,22
31030,94902,129324,2.5,2,10,84,1,73,35,64,...,100,0,1,0,0.500000,0.000000,1,1.000000,87,14
32385,99007,131015,2.5,3,10,187,3,72,30,69,...,63,0,2,2,0.666667,0.666667,1,0.333333,1232,16


Trying rounding the data to build clear classifier & regressor

In [197]:
params = {}
tot_ce=[]

param ={
    'colsample_bytree': 0.5,
    'min_child_weight': 6.0, 
    'n_estimators': 630.0,
    'subsample': 0.9500000000000001,
    'eta': 0.025, 
    'max_depth': 13.0, 
    'gamma': 0.6000000000000001,
    'eval_metric': 'rmse',
    'objective': 'reg:linear',
    'nthread' : 5,
    'silent' : 1,
    'lambda': 0.8500000000000001,
    'alpha' : 0.75
}
   
dtrain = xgb.DMatrix(X_train, y_train.round())
dvalid = xgb.DMatrix(X_test, y_test.round())    

num_round = int(param['n_estimators'])
del param['n_estimators']
plst = list(param.items())

model = xgb.train(plst, dtrain, num_round)
predictions = model.predict(dvalid)#.reshape((X_test.shape[0], 3))
score = mean_squared_error(y_test, predictions)**0.5 #0.20764286789333591

In [198]:
score

0.4601301126965906

In [199]:
0.20764286789333591**0.5

0.45567846985932514

In [196]:
y_test

array([ 1.67,  1.67,  3.  , ...,  1.33,  1.67,  1.33])

In [221]:
import whoosh as wh

In [253]:
pd.DataFrame(y_test.round()).to_csv(path['data']+'actual.csv')  #, predictions.round()

In [254]:
pd.DataFrame(predictions.round()).to_csv(path['data']+'pred.csv')

In [255]:
import nltk;